In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from preprocess import AnomalyDetectionDataset, device_change
from model import AutoencoderAnomalyDetection, TrainEvaluation
from torch import nn

In [3]:
first_box_dir = '../anomaly detection/first_box_split'
second_box_dir = '../anomaly detection/second_box_split'
third_box_dir = '../anomaly detection/third_box_split'
background_dir = '../anomaly detection/background_split'

filepath_first_box = f'{first_box_dir}/first_box_*.csv'
filepath_second_box = f'{second_box_dir}/second_box_*.csv'
filepath_third_box = f'{third_box_dir}/third_box_*.csv'
filepath_background = f'{background_dir}/background_0.csv'

### Getting Data with pytorch

In [4]:
device = device_change()

Using mps device


In [5]:
background = AnomalyDetectionDataset(filepath_background, three_channels=True)

### Model Implementation

In [7]:
learning_rate = 1e-3
batch_size = 32
epochs = 5

In [8]:
train_size = int(len(background) * 0.8)
test_size = len(background) - train_size
valid_size = train_size * .15

In [9]:
torch.manual_seed(42)
data = DataLoader(background, batch_size=batch_size, shuffle=True)

In [10]:
X_train_full, X_test= random_split(background, [train_size, test_size])

ValueError: all input arrays must have the same shape

In [ ]:
model = AutoencoderAnomalyDetection().to(device)
print(model)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_eval = TrainEvaluation(data, data, model, loss_func, optimizer, epochs, batch_size=batch_size)

AutoencoderAnomalyDetection(
  (encoder): Sequential(
    (0): Flatten(start_dim=128, end_dim=256)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=67200, bias=True)
    (7): Unflatten(dim=1, unflattened_size=[3, 32, 700])
  )
  (decoder): Sequential(
    (0): Flatten(start_dim=128, end_dim=256)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=67200, bias=True)
    (7): Unflatten(dim=1, unflattened_size=[3, 32, 700])
  )
)


In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_eval.train_loop()
    train_eval.test_loop()
print("Done!")

: 